## What is a Conda Environment?
A Conda environment is an isolated, self-contained directory that contains all the necessary dependencies—such as specific versions of Python, libraries, and tools—needed for a particular project or use case. These environments are created and managed using Conda, an open-source package and environment management system, which is often used in data science and software development to manage dependencies easily.

A Conda environment has:

- Python version (or potentially a different programming language runtime)
- Libraries required for the project (like pandas, numpy, scikit-learn, etc.)
- Scripts or other tools required for the project.

### Create a conda environment

- Create a conda envionment 

```
conda create -n mlops python=3.9
```

- List conda environments

```
conda env list
```

- Activate the environment

```
conda activate mlops
```

- Print conda environment information

```
conda info
```

- Deactivate the environment

```
conda deactivate

```





### Installing and Configuring mlflow

- Install mlflow library
```
pip install mlflow
```

- Start the mlflow server

```
mlflow server --host 127.0.0.1 --port 8080
```


### Import required libraries

In [14]:
import mlflow
from mlflow.models import infer_signature

import numpy as np
import pandas as pd
import requests
import base64
from getpass import getpass
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [3]:
cars_df = pd.read_csv("used_car.csv")

In [4]:
x_columns = ['KM_Driven', 'Fuel_Type', 'age',
              'Transmission', 'Owner_Type', 'Seats',
              'make', 'mileage_new', 'engine_new', 'model',
              'power_new', 'Location']

In [5]:
cars_df = cars_df[x_columns + ['Price']].dropna()

In [6]:
cat_features = ['Fuel_Type',
                'Transmission', 'Owner_Type', 'model',
                'make', 'Location']

In [7]:
num_features = list(set(x_columns) - set(cat_features))

In [8]:
train, test = train_test_split(cars_df,
                               train_size = 0.8,
                               random_state = 100)

In [9]:
train.shape

(2472, 13)

In [10]:
test.shape

(619, 13)

In [11]:
train.to_parquet("train.parquet")

In [12]:
test.to_parquet("test.parquet")

### Version Control train and test datasets on Github

Useful commands for github

```
git clone https://github.com/username/mlopsdemo.git
cd mlopsdemo
cp /path/to/your/mlopsdemo/ .
git add train.parquet
git commit -m "Initial commit of dataset"
git push origin main
```


In [18]:
train_df = pd.read_parquet("https://raw.githubusercontent.com/manaranjanp/MLOpsDemo/main/datasets/train.parquet")

In [19]:
train_df.head(5)

KM_Driven Fuel_Type  age Transmission Owner_Type  Seats      make  \
2954         65    Diesel    6       Manual      First    5.0   hyundai   
716          42    Petrol    5       Manual      First    5.0   hyundai   
250          31    Petrol    3       Manual      First    5.0    maruti   
1506         42    Petrol    5    Automatic      First    5.0     honda   
859         169    Diesel    5       Manual      First    7.0  mahindra   

      mileage_new  engine_new  model  power_new   Location  Price  
2954        24.00        1120  grand      70.00      Delhi   3.99  
716         18.60        1197    i20      81.83     Mumbai   5.05  
250         20.40        1197  swift      81.80      Kochi   5.75  
1506        15.50        1198  amaze      86.70       Pune   4.34  
859         14.95        2489   xylo      93.70  Hyderabad   5.00

In [21]:
test_df = pd.read_parquet("https://raw.githubusercontent.com/manaranjanp/MLOpsDemo/main/datasets/test.parquet")

### Create the ML Pipeline

In [23]:
numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[('onehot', 
                                           OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_features),                  
        ('cat', categorical_transformer, cat_features),
    ])

params = { "n_estimators": 400,
           "max_depth": 4 }

xgb_regressor = GradientBoostingRegressor(**params)

reg = Pipeline(steps=[('preprocessor', preprocessor),
                      ('regressor', xgb_regressor)])           

reg.fit(train_df[x_columns], 
        train_df['Price'])

rmse = np.sqrt(mean_squared_error(test_df['Price'], 
                                  reg.predict(test_df[x_columns])))
r2 = r2_score(test_df['Price'], reg.predict(test_df[x_columns]))

### Track Experiments

In [24]:
# Set our tracking server uri for logging
mlflow.set_tracking_uri(uri="http://127.0.0.1:9090")

# Create a new MLflow Experiment
mlflow.set_experiment("UserCarPrice Experiment V1")

# Start an MLflow run
with mlflow.start_run():
    # Log the hyperparameters
    mlflow.log_params(params)

    # Log the loss metric
    mlflow.log_metric("rmse", rmse)
    # Log the loss metric
    mlflow.log_metric("r2", r2)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", "Gradient Boosting Model")

    # Infer the model signature
    signature = infer_signature(train_df[x_columns], 
                                reg.predict(train_df[x_columns]))

    # Log the model
    model_info = mlflow.sklearn.log_model(
        sk_model=reg,
        artifact_path="usedcarmodel",
        signature=signature,
        input_example=train_df[x_columns],
        registered_model_name="userd_car_model",
    )

2024/11/22 12:08:08 INFO mlflow.tracking.fluent: Experiment with name 'UserCarPrice Experiment V1' does not exist. Creating a new experiment.
/Users/manaranjanp/anaconda3/envs/mlops/lib/python3.9/site-packages/mlflow/types/utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
Registered model 'userd_car_model' already exists. Creating a new version of t

🏃 View run whimsical-goose-655 at: http://127.0.0.1:9090/#/experiments/544501832851099702/runs/c108557530b24ad2894964dae4cde86d
🧪 View experiment at: http://127.0.0.1:9090/#/experiments/544501832851099702


Created version '2' of model 'userd_car_model'.


In [25]:
model_info.model_uri

'runs:/c108557530b24ad2894964dae4cde86d/usedcarmodel'